# Read AVI Files

This notebook showcases how to read the .avi depth video files in the dataset.

In [ ]:
from fsspec import filesystem
import cv2
import tempfile
from dandi.dandiapi import DandiAPIClient
from tqdm.notebook import tqdm

def stream_avifile(DANDISET_ID, file_path, temp_file, num_chunks=None):
    '''Stream .avi file from DANDI archive.
    
    Parameters
    ----------
    DANDISET_ID : str
        Dandiset ID
    file_path : str
        Path to .avi file in DANDI archive
    temp_file : file-like object
        Temporary file to write the .avi file to
    num_chunks : int, optional
        Number of chunks to read from the file. If None, read the whole file.
    '''
    with DandiAPIClient() as client:
        asset = client.get_dandiset(DANDISET_ID, 'draft').get_asset_by_path(file_path)
        s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)
    fs = filesystem("http")
    file_system = fs.open(s3_url, "rb")
    file_size = file_system.size
    chunk_size = 10_000_000 # 10 MB
    num_chunks = num_chunks or file_size // chunk_size
    for _ in tqdm(range(num_chunks)):
        chunk = file_system.read(length=chunk_size)
        temp_file.write(chunk)

In [ ]:
def show_video(file_path: str):
    '''Show video in a window.

    Parameters
    ----------
    file_path : str
        Path to video file
    '''
    raw_video = cv2.VideoCapture(file_path)

    while True:
        # Read a frame from the video
        ret, frame = raw_video.read()

        # If the frame is not read successfully, the video has ended
        if not ret:
            break

        frame = frame / frame.max() # Normalize the frame

        # Display the frame in a window
        cv2.imshow('Frame', frame)

        # Wait for a key press to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # Release the video capture and close the window
    raw_video.release()
    cv2.destroyAllWindows()

In [ ]:
DANDISET_ID = '000559'
avi_file_path = 'sub-10/sub-10_ses-reinforcement-0e0aca4f-6a99-46eb-9eb8-439f36e66c40_behavior+image+ogen/f9af8a18-7cab-471c-802f-6937edeaec68_external_file_0.avi'
num_chunks = 10 # Use None to read the whole file

with tempfile.NamedTemporaryFile() as temp:
    stream_avifile(DANDISET_ID, avi_file_path, temp, num_chunks=num_chunks)
    temp.flush()
    show_video(temp.name)